##### 版權所有 2024 Google LLC.


In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini API：以 REST 呼叫函式

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/rest/Function_calling_REST.zh.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
</table>


此筆記本提供了快速程式碼範例，供你瞭解如何使用 `curl` 開始進行函式呼叫。

你可以在 Google Colab 中執行此操作，或將 `curl` 指令複製/貼到你的終端機中。

若要執行此筆記本，必須將你的 API 金鑰儲存在名為 GOOGLE_API_KEY 的 Colab Secret 中。如果你是在不同環境中執行，則可以將你的金鑰儲存在環境變數中。請參閱 [驗證](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) 以瞭解更多資訊。


In [ ]:
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

##函式呼叫如何運作


函式呼叫讓開發人員可以在程式碼中建立函式描述，然後將描述傳遞給請求中的語言模型。模型回應包含與描述匹配的函式名稱，以及呼叫函式的引數。函式呼叫可讓你在生成式 AI 應用程式中將函式當作工具使用，且你可以在單一請求中定義多個函式。函式呼叫會回傳 JSON，其中包含函式名稱及要在程式碼中使用的引數。

函式使用「函式宣告」描述。將函式宣告列表傳遞給語言模型的查詢後，模型會以與 [OpenAPI 相容的 Schema](https://spec.openapis.org/oas/v3.0.3#schema) 格式相同的物件進行回應，其中包含函式名稱及其引數，並嘗試使用回傳的其中一個函式回答使用者查詢。語言模型會透過分析函式宣告來了解函式的目的。模型並不會實際呼叫函式。相反地，開發人員會使用回應中的 [OpenAPI 相容的 Schema](https://spec.openapis.org/oas/v3.0.3#schema) 物件呼叫模型回傳的函式。

實作函式呼叫時，你要建立一個或多個「函式宣告」，然後將函式宣告加入傳遞給模型的 `tools` 物件。每個函式宣告都包含下列函式資訊：

* 函式名稱
* 使用 [OpenAPI 相容的 Schema](https://spec.openapis.org/oas/v3.0.3#schemawr) 格式的函式參數。支援 [選取子集](https://ai.google.dev/api/rest/v1beta/Tool#Schema)。使用 curl 時，使用 JSON 指定 schema。
* 函式描述 (選用)。為了獲得最佳結果，我們建議你納入描述。

本筆記包含與 `GenerativeModel` 類別及其方法共同進行 REST 呼叫的 curl 範例。


## 支援的模型


以下模型支援函式呼叫：

* `gemini-pro`


## 函式呼叫 cURL 範例

當你使用 cURL 時，功能和參數資訊會包含在
`tools` 元素中。`tools` 元素中的每個功能宣告包含
函式名稱，使用
[OpenAPI 相容架構](https://spec.openapis.org/oas/v3.0.3#schema) 指定的參數以及函式描述。下列範例展示如何使用具有函式呼叫的 curl 指令：


### 單次捲曲範例

單次處理是指你一次呼叫語言模型。透過呼叫函式，單次處理使用案例可能是當你提供模型一個自然語言查詢和一個函式清單。這種情況下，模型會使用包含函式名稱、參數和說明的函式宣告，來預測要呼叫的函式以及使用哪個引數來呼叫它。

下列捲曲範例是一個傳遞關於電影正在播放地點相關資訊的函式說明範例。請求中包含數個函式宣告，例如「`find_movies`」和「`find_theaters`」。


In [6]:
%%bash

curl "https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?key=$GOOGLE_API_KEY" \
  -H 'Content-Type: application/json' \
  -d '{
    "contents": {
      "role": "user",
      "parts": {
        "text": "Which theaters in Mountain View show Barbie movie?"
    }
  },
  "tools": [
    {
      "function_declarations": [
        {
          "name": "find_movies",
          "description": "find movie titles currently playing in theaters based on any description, genre, title words, etc.",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
              },
              "description": {
                "type": "string",
                "description": "Any kind of description including category or genre, title words, attributes, etc."
              }
            },
            "required": [
              "description"
            ]
          }
        },
        {
          "name": "find_theaters",
          "description": "find theaters based on location and optionally movie title which are is currently playing in theaters",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
              },
              "movie": {
                "type": "string",
                "description": "Any movie title"
              }
            },
            "required": [
              "location"
            ]
          }
        },
        {
          "name": "get_showtimes",
          "description": "Find the start times for movies playing in a specific theater",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
              },
              "movie": {
                "type": "string",
                "description": "Any movie title"
              },
              "theater": {
                "type": "string",
                "description": "Name of the theater"
              },
              "date": {
                "type": "string",
                "description": "Date for requested showtime"
              }
            },
            "required": [
              "location",
              "movie",
              "theater",
              "date"
            ]
          }
        }
      ]
    }
  ]
}' 2> /dev/null

{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "functionCall": {
              "name": "find_theaters",
              "args": {
                "movie": "Barbie",
                "location": "Mountain View, CA"
              }
            }
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "index": 0,
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE"
        }
      ]
    }
  ],
  "promptFeedback": {
    "safetyRatings": [
      {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "pro

### 多輪 curl 範例

你可以藉由執行以下動作，實作多輪函式呼叫情境：

1. 呼叫語言模型以取得函式呼叫回應。這是第一輪。
1. 使用第一輪的函式呼叫回應和從函式中取得的函式回應，呼叫語言模型。這是第二輪。

第二輪的回應，會概括結果以回答你在第一輪中的疑問，或包含可用於取得更多資訊的第二個函式呼叫。

本主題包含兩個多輪 curl 範例：

* 使用前一輪函式回應的 curl 範例
* 多次呼叫語言模型的 curl 範例


#### 使用先前回合回應的 Curl 範例

以下 curl 範例會呼叫由前一個單回合範例回傳的函式和參數，以取得回應。由單回合範例回傳的方法和參數會包含在此 JSON 中。

```json
"functionCall": {
  "name": "find_theaters",
  "args": {
    "movie": "Barbie",
    "location": "Mountain View, CA"
  }
}
```


In [7]:
%%bash

curl "https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?key=$GOOGLE_API_KEY" \
  -H 'Content-Type: application/json' \
  -d '{
    "contents": [{
      "role": "user",
      "parts": [{
        "text": "Which theaters in Mountain View show Barbie movie?"
    }]
  }, {
    "role": "model",
    "parts": [{
      "functionCall": {
        "name": "find_theaters",
        "args": {
          "location": "Mountain View, CA",
          "movie": "Barbie"
        }
      }
    }]
  }, {
    "role": "function",
    "parts": [{
      "functionResponse": {
        "name": "find_theaters",
        "response": {
          "name": "find_theaters",
          "content": {
            "movie": "Barbie",
            "theaters": [{
              "name": "AMC Mountain View 16",
              "address": "2000 W El Camino Real, Mountain View, CA 94040"
            }, {
              "name": "Regal Edwards 14",
              "address": "245 Castro St, Mountain View, CA 94040"
            }]
          }
        }
      }
    }]
  }],
  "tools": [{
    "functionDeclarations": [{
      "name": "find_movies",
      "description": "find movie titles currently playing in theaters based on any description, genre, title words, etc.",
      "parameters": {
        "type": "OBJECT",
        "properties": {
          "location": {
            "type": "STRING",
            "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
          },
          "description": {
            "type": "STRING",
            "description": "Any kind of description including category or genre, title words, attributes, etc."
          }
        },
        "required": ["description"]
      }
    }, {
      "name": "find_theaters",
      "description": "find theaters based on location and optionally movie title which are is currently playing in theaters",
      "parameters": {
        "type": "OBJECT",
        "properties": {
          "location": {
            "type": "STRING",
            "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
          },
          "movie": {
            "type": "STRING",
            "description": "Any movie title"
          }
        },
        "required": ["location"]
      }
    }, {
      "name": "get_showtimes",
      "description": "Find the start times for movies playing in a specific theater",
      "parameters": {
        "type": "OBJECT",
        "properties": {
          "location": {
            "type": "STRING",
            "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
          },
          "movie": {
            "type": "STRING",
            "description": "Any movie title"
          },
          "theater": {
            "type": "STRING",
            "description": "Name of the theater"
          },
          "date": {
            "type": "STRING",
            "description": "Date for requested showtime"
          }
        },
        "required": ["location", "movie", "theater", "date"]
      }
    }]
  }]
}' 2> /dev/null

{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "OK. I found two theaters in Mountain View that are showing the Barbie movie: AMC Mountain View 16 and Regal Edwards 14."
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "index": 0,
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE"
        }
      ]
    }
  ]
}


#### Curl 範例重複呼叫語言模型

以下 curl 範例重複呼叫語言模型以呼叫函式。每次模型呼叫函式時，它都可以使用不同的函式來回答請求中的不同使用者查詢。


In [9]:
%%bash

curl "https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent?key=$GOOGLE_API_KEY" \
  -H 'Content-Type: application/json' \
  -d '{
    "contents": [{
      "role": "user",
      "parts": [{
        "text": "Which theaters in Mountain View show Barbie movie?"
    }]
  }, {
    "role": "model",
    "parts": [{
      "functionCall": {
        "name": "find_theaters",
        "args": {
          "location": "Mountain View, CA",
          "movie": "Barbie"
        }
      }
    }]
  }, {
    "role": "function",
    "parts": [{
      "functionResponse": {
        "name": "find_theaters",
        "response": {
          "name": "find_theaters",
          "content": {
            "movie": "Barbie",
            "theaters": [{
              "name": "AMC Mountain View 16",
              "address": "2000 W El Camino Real, Mountain View, CA 94040"
            }, {
              "name": "Regal Edwards 14",
              "address": "245 Castro St, Mountain View, CA 94040"
            }]
          }
        }
      }
    }]
  },
  {
    "role": "model",
    "parts": [{
      "text": " OK. Barbie is showing in two theaters in Mountain View, CA: AMC Mountain View 16 and Regal Edwards 14."
    }]
  },{
    "role": "user",
    "parts": [{
      "text": "Can we recommend some comedy movies on show in Mountain View?"
    }]
  }],
  "tools": [{
    "functionDeclarations": [{
      "name": "find_movies",
      "description": "find movie titles currently playing in theaters based on any description, genre, title words, etc.",
      "parameters": {
        "type": "OBJECT",
        "properties": {
          "location": {
            "type": "STRING",
            "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
          },
          "description": {
            "type": "STRING",
            "description": "Any kind of description including category or genre, title words, attributes, etc."
          }
        },
        "required": ["description"]
      }
    }, {
      "name": "find_theaters",
      "description": "find theaters based on location and optionally movie title which are is currently playing in theaters",
      "parameters": {
        "type": "OBJECT",
        "properties": {
          "location": {
            "type": "STRING",
            "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
          },
          "movie": {
            "type": "STRING",
            "description": "Any movie title"
          }
        },
        "required": ["location"]
      }
    }, {
      "name": "get_showtimes",
      "description": "Find the start times for movies playing in a specific theater",
      "parameters": {
        "type": "OBJECT",
        "properties": {
          "location": {
            "type": "STRING",
            "description": "The city and state, e.g. San Francisco, CA or a zip code e.g. 95616"
          },
          "movie": {
            "type": "STRING",
            "description": "Any movie title"
          },
          "theater": {
            "type": "STRING",
            "description": "Name of the theater"
          },
          "date": {
            "type": "STRING",
            "description": "Date for requested showtime"
          }
        },
        "required": ["location", "movie", "theater", "date"]
      }
    }]
  }]
}'  2> /dev/null

{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "functionCall": {
              "name": "find_movies",
              "args": {
                "location": "Mountain View, CA",
                "description": "comedy"
              }
            }
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "index": 0,
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE"
        }
      ]
    }
  ],
  "promptFeedback": {
    "safetyRatings": [
      {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        